<a href="https://colab.research.google.com/github/netrapathak/dataria-capstone/blob/master/Copy_of_PitchBook_Code_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import os
from cleanco import cleanco
import re
import unicodedata
!pip install ftfy
from ftfy import fix_text
import sys
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
!pip install sparse_dot_topn 
import sparse_dot_topn.sparse_dot_topn as ct

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [0]:
os.getcwd()

'/Users/yash/Downloads'

In [0]:
pitch = pd.read_csv('Pitchbook.csv', dtype='unicode')
uspto = pd.read_csv('uspto.csv', dtype='unicode')

# Removing legal suffixes from comapnies using cleanco.

In [0]:
ser = []
for company in pitch['Company Name']:
    company_name = cleanco(company)
    ser.append(company_name.clean_name())
pitch['Cleaned Company'] = ser

,organization,Company
0,"Sound Sources Technology, Inc.",Sound Sources Technology
1,"Amkor Technology, Inc.",Amkor Technology
2,"GM Global Technology Operations, Inc.",GM Global Technology Operations
3,Elwha LLC,Elwha
4,Unity Semiconductor Corporation,Unity Semiconductor
...,...,...
453685,Halkey-Roberts Corporation,Halkey-Roberts
453686,"Gardner Denver, Inc.",Gardner Denver
453687,Resilient Network Systems Inc,Resilient Network Systems
453688,"Vermont Herb & Spice Accessories, Inc.",Vermont Herb & Spice Accessories


In [0]:
li = []
for company in uspto['organization']:
    company_name = cleanco(str(company))
    li.append(company_name.clean_name())
uspto['Company'] = li

In [0]:
uspto = pd.DataFrame(uspto)

In [0]:
uspto.head(20) 

,organization,Company
0,"Sound Sources Technology, Inc.",Sound Sources Technology
1,"Amkor Technology, Inc.",Amkor Technology
2,"GM Global Technology Operations, Inc.",GM Global Technology Operations
3,Elwha LLC,Elwha
4,Unity Semiconductor Corporation,Unity Semiconductor
5,"United Parcel Service of America, Inc.",United Parcel Service of America
6,"HOLTEC INTERNATIONAL, INC.",HOLTEC INTERNATIONAL
7,Wisconsin Alumni,Wisconsin Alumni
8,"Nexteer (Beijing) Technology Co., Ltd.",Nexteer (Beijing) Technology Co
9,Aristocrat Technologies Australia Pty Limited,Aristocrat Technologies Australia Pty


Merging both datasets with no company suffixes.

In [0]:
res = pd.merge(pitch,uspto,left_on='Cleaned Company',right_on='Company')
left = pitch['Cleaned Company']
right = uspto['Company']

In [0]:
res

,Company ID,Company Name,Company Former Name,Description,Financing Status Note,Primary Industry Sector,Primary Industry Group,Primary Industry Code,Verticals,All Industries,...,HQ Phone,HQ Fax,HQ Email,Primary Contact,Primary Contact Title,Secondary Contact,PitchBook Link,Cleaned Company,organization,Company
0,61318-09,20/20 GeneSystems,NaN,Developer of commercial blood tests for the ea...,"The company has raised $213,778 of angel fundi...",Healthcare,Pharmaceuticals and Biotechnology,Biotechnology,"Life Sciences, Oncology",Biotechnology*; Laboratory Services (Healthcar...,...,+1 (240) 453-6339,+1 (240) 403-0289,investors@2020gene.com,Jonathan Cohen,"Founder, Chief Executive Officer & President",NaN,View Company Online,20/20 GeneSystems,"20/20 GeneSystems, Inc.",20/20 GeneSystems
1,125595-28,8th Continent,NaN,Producer of soy milk. The company offers glute...,The company as acquired by Stremicks Heritage ...,Consumer Products and Services (B2C),Consumer Non-Durables,Beverages,NaN,Beverages*; Food Products,...,+1 (800) 247-6458,NaN,NaN,NaN,NaN,NaN,View Company Online,8th Continent,"8th Continent, LLC",8th Continent
2,12761-02,A&G Pharmaceutical,NaN,Developer of personalized medicine products de...,The company received $3.34 million of grant fu...,Healthcare,Pharmaceuticals and Biotechnology,Drug Discovery,"Life Sciences, Oncology",Drug Discovery*; Other Pharmaceuticals and Bio...,...,+1 (410) 884-4100,+1 (410) 884-1607,info@agpharma.com,Ginette Serrero,"Co-Founder, Chief Executive Officer & Board Me...",NaN,View Company Online,A&G Pharmaceutical,"A&G Pharmaceutical, Inc.",A&G Pharmaceutical
3,56386-09,Abeome,NaN,Developer of a prescription product to help in...,The company received $1.48 million of grant fu...,Healthcare,Pharmaceuticals and Biotechnology,Biotechnology,"Life Sciences, Oncology",Biotechnology*; Drug Discovery,...,+1 (706) 542-7889,NaN,info@abeomecorp.com,Richard Shimkets,"Chief Executive Officer, Board Member & President",NaN,View Company Online,Abeome,Abeome Corporation,Abeome
4,58047-67,AbGenomics International,NaN,Developer of antibody therapeutics designed fo...,The company raised $16 million of venture fund...,Healthcare,Pharmaceuticals and Biotechnology,Biotechnology,"Life Sciences, Oncology",Biotechnology*; Drug Delivery; Drug Discovery,...,NaN,NaN,NaN,Rong-Hwa Lin,"Chief Executive Officer, Founder & Chairman",NaN,View Company Online,AbGenomics International,AbGenomics International Inc.,AbGenomics International
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5306,268616-08,Terralogix,NaN,Developer of an artificial intelligence based ...,"The company raised GBP 300,010 of venture fund...",Information Technology,Software,Social/Platform Software,Artificial Intelligence & Machine Learning,Social/Platform Software*; Business/Productivi...,...,NaN,NaN,info@terralogix.ai,Arthur Hughes,Chief Executive Officer & Chairman,NaN,View Company Online,Terralogix,Terralogix Inc.,Terralogix
5307,265190-41,Tournicare,NaN,Manufacturer of a non-circumferential blood pr...,The company raised venture funding from Artesi...,Healthcare,Healthcare Devices and Supplies,Monitoring Equipment,Manufacturing,Monitoring Equipment*,...,+61 (0)13 0076 3130,NaN,NaN,Niels van Sparrentak,Co-Founder and Chief Executive Officer,NaN,View Company Online,Tournicare,Tournicare Pty Ltd,Tournicare
5308,266383-09,Trace,NaN,Developer of an enterprise platform intended t...,The company raised an undisclosed amount of se...,Information Technology,Software,Business/Productivity Software,FinTech,Business/Productivity Software*,...,NaN,NaN,NaN,Mike Gonzalez,Co-Founder & Chief Executive Officer,NaN,View Company Online,Trace,"Trace, Inc.",Trace
5309,266384-80,UMA,NaN,Developer of a decentralized financial contrac...,The company raised $4 million of seed funding ...,Information Technology,Software,Financial Software,"Cryptocurrency/Blockchain, FinTech",Financial Software*,...,+1 (415) 868-3978,NaN,NaN,Allison Lu,Co-Found

# Fuzzy Matching Algorithm using ngrams and tf-idf vectorizer.

In [0]:
def ngrams(string, n=3):
    string = fix_text(string) 
    string = string.encode("ascii", errors="ignore").decode() 
    string = string.lower() 
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string) 
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title() 
    string = re.sub(' +',' ',string).strip()
    string = ' '+ string +' ' 
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

company_names = uspto['organization']

vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(company_names.values.astype('U'))

In [0]:
company = pitch['Company Name']
tf_idf_matrix_alt = vectorizer.fit_transform(company.values.astype('U'))

In [0]:
company_uspto = uspto['Company']
tf_idf_matrix_uspto = vectorizer.fit_transform(company_uspto.values.astype('U'))

In [0]:
len(company)

86440

Cosine similarity function.

In [0]:
def cossim_top(A, B, ntop, lower_bound=0):
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)
    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)
    return csr_matrix((data,indices,indptr),shape=(M,N))

In [0]:
import time
t1 = time.time()
matches = cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.8)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 1843.9344069957733


In [0]:
t1 = time.time()
matches_new = cossim_top(tf_idf_matrix_alt, tf_idf_matrix_alt.transpose(), 1, 0.7)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 9.387048959732056


In [0]:
t1 = time.time()
matches_alt = cossim_top(tf_idf_matrix_uspto, tf_idf_matrix_uspto.transpose(), 10, 0.7)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 1152.9547078609467


In [0]:
def get_len_vector(name_vector): 
    return len(name_vector)
def get_matches_df(sparse_matrix, name_vector, top=10000):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})

In [0]:
matches_df = get_matches_df(matches, company_names)
matches_df = matches_df[matches_df['similairity'] > 0.80] # Remove all exact matches
matches_df.sample(50)
#matches_df.shape

,left_side,right_side,similairity
286,Honeywell International Inc.,oHoneywell International Inc.,0.802247
9330,Westalische Metall Industrie KG Hueck & Co.,Westfalische Metall Industrie KG Huec & Co.,0.822399
3879,Penn Fishing Tackle Manufacturing Company,Penn Fishing Tackle Manufacturing Company,1.000000
3373,"PRECISION COATING INNOVATIONS, LLC","PRECISION COATING INNOVATIONS, LLC",1.000000
878,"PIRAMAL GLASS—USA, INC.","PIRAMAL GLASS—USA, INC.",1.000000
7810,Racla Co. Limited,Racla Co. Limited,1.000000
9727,Specialized Banking Furniture International,Specialized Banking Furniture International,1.000000
8493,Lateral Laboratories Limited,Lateral Laboratories Limited,1.000000
6715,FRENI BREMBO S.P.A.,FRENI BREMBO S.P.A.,1.000000
4091,Saint-Gobain Performance Plastics Corporation,Saint-Gobain Performance Plastics Rencol Limited,0.828120


In [0]:
uspto_df = get_matches_df(matches_new, company_uspto)
uspto_df = matches_df[matches_df['similairity'] > 0.70] # Remove all exact matches
uspto_df.sample(50)
#matches_df.shape

,left_side,right_side,similairity
4227,The Board of Trustees of the University of Ark...,Board of Trutees of the University of Arkansas,0.933145
2000,Hyundai Motor Corporation,Hyundai Motor Corporation,1.000000
2972,RCD Technology Inc,RCD Technology Inc,1.000000
983,"GoFormz, Inc.","GoFormz, Inc.",1.000000
2049,Videojet Technologies Inc,Videojet Technoogies Inc.,0.869231
3688,GM Global Technology Operations Inc.,GM Global Technology Operations LLC,0.957633
7922,"MTM ROBOTICS, LLC","MTM ROBOTICS, LLC",1.000000
1113,King Technology,King Technology,1.000000
7425,"Quick Hit, Inc.","Quick Hit, Inc.",1.000000
314,Research In Motion Limited,Reseach In Motion Limited,0.855800


In [0]:
uspto_df = get_matches_df(matches_alt, company_uspto)
uspto_df = matches_df[matches_df['similairity'] > 0.70] # Remove all exact matches
uspto_df.sample(50)
#matches_df.shape

,left_side,right_side,similairity
5496,Industrial Design Laboratories Inc,Industrial Design Laboratories Inc.,1.000000
1386,The University of North Carolina at Charlotte,UNIVERSITY OF NORTH CAROLINA CHARLOTTE,0.890796
1695,Avago Technologies Wireless IP (Singapore) Pte...,Avago Technologies General IP (Singapore) Pte Ltd,0.806132
3572,"Applied Materials, Inc.",Applied Materials,0.976525
2780,"Silicon Optronics, Inc.","SILICON OPTONICS, INC.",0.816680
2236,"Sercel, Inc.","Sercel, Inc.",1.000000
6000,Realtek Semiconductor,Realtek Semiconductor,1.000000
4160,"Semiconductor Components Industries, L.L.C.","SEMICONDUCTOR COMPONENT INDUSTRIES, LLC",0.943045
315,Research In Motion Limited,Researc In Motion Limited,0.853749
2212,"South Cone, Inc.","South Cone, Inc.",1.000000


NameError: ignored

In [0]:
org_name_clean = pitch['Company Name'].unique()
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
tfidf = vectorizer.fit_transform(org_name_clean.astype('U'))
print('Vecorizing completed...')
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)
org_column = uspto['organization'].unique() #column to match against in the messy data

unique_org = set(org_column.astype('U')) 

def getNearestN(query):
    queryTFIDF_ = vectorizer.transform(query)
    distances, indices = nbrs.kneighbors(queryTFIDF_)
    return distances, indices

import time
t1 = time.time()
print('getting nearest n...')
distances, indices = getNearestN(unique_org)
t = time.time()-t1
print("COMPLETED IN:", t)
unique_org = list(unique_org) 


Vecorizing completed...
getting nearest n...
COMPLETED IN: 1446.7034108638763
finding matches...
Building data frame...
Done


In [0]:
matches = []
for i,j in enumerate(indices):
    temp = [round(distances[i][0],2), pitch.values[j][0][1],unique_org[i]]
    matches.append(temp)
print('Building data frame...')  
matches = pd.DataFrame(matches, columns=['Match confidence (lower is better)','Matched name','Original name'])
print('Done')

Building data frame...
Done


In [0]:
matches['Match confidence (lower is better)']

0         1.10
1         1.05
2         1.05
3         0.77
4         1.01
          ... 
453685    0.93
453686    0.89
453687    0.86
453688    0.87
453689    1.04
Name: Match confidence (lower is better), Length: 453690, dtype: float64

In [0]:
match = matches.sort_values(by=['Match confidence (lower is better)'])

In [0]:
match_1 = match[match['Match confidence (lower is better)'] < 0.79]

In [0]:
match_1

,Match confidence (lower is better),Matched name,Original name
231702,0.00,Cuseum,CUSEUM
368331,0.00,ALCOM,ALCOM
5538,0.00,SOFIE,Sofie
328657,0.00,INNOVIA,INNOVÃÂ¢ÃÂÃÂ²IA
292139,0.00,Famoco,FAMOCO
...,...,...,...
126956,0.78,Unlimited,"Bath Unlimited, Inc."
236244,0.78,Interactive Technologies (wireless service),INTERACTIVE WIRELESS TECHNOLOGIES LLC
127275,0.78,Evolution Energy Technologies,Evolution Technologies Inc.
152325,0.78,Alternative Fuels Group,"Alternate Fuels, Inc."
